In [20]:
import requests
import pandas as pd
import tiktoken
# Replace 'your_file.xlsx' with the path to your Excel file
file_path = 'directory_data.xlsx'
df = pd.read_excel(file_path);


/Users/mrohi233/marriott/python_projects/mlenv/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [22]:
# List of columns to keep
columns_to_keep = ['CI ID','Name', 'Alias','Approval group','Managed by','Support group','Manufacturer/Vendor','Short Description','Assigned to','Business criticality',
                  'Class','Comments','Consumer type','Created by','Domain','Managed By Group','Parent','Problem Manager Group','Problem Owner Group','Product Owner Executive','Product Owner'
                  ,'Revenue Impact','Service Executive ','Service Owner','Service Team','Supported by','Updated by'];

# Filter the DataFrame to include only the specified columns
df = df[columns_to_keep]
df['CI ID'] = df['CI ID'].astype(str)
df.fillna("", inplace=True)  # Fill NaN with "Unknown"
# Create combined column
df["Text"] = df[columns_to_keep].apply(
    lambda row: "; ".join(f"{col}: {str(row[col]).strip()[:100]}" for col in columns_to_keep),
    axis=1
)
df['Text'] = df['Text'].astype(str)
print(df['Text'])
rows_count = df.shape[0]
print("Number of rows in the DataFrame:", rows_count)

encoding = tiktoken.get_encoding("cl100k_base")
df["n_tokens"] = df.Text.apply(lambda x: len(encoding.encode(x)))



0       CI ID: CI000007819; Name: Fidelity Retirement ...
1       CI ID: CI000539436; Name: 4Sight Budgeting and...
2       CI ID: CI000553739; Name: Above Property LAN S...
3       CI ID: CI084125824; Name: Accertify - Fraud an...
4       CI ID: BSN0002761; Name: Accessories/Periphera...
                              ...                        
2242    CI ID: CI076976718; Name: Workstation - Mac; A...
2243    CI ID: CI076976711; Name: Workstation - Micros...
2244    CI ID: CI076976710; Name: Workstation - Micros...
2245    CI ID: CI076976701; Name: Workstation - Toshib...
2246    CI ID: CI007606104; Name: Workstation Imaging ...
Name: Text, Length: 2247, dtype: object
Number of rows in the DataFrame: 2247


/var/folders/rp/sm11qb895w7_c9y7d3nfkxz00000gr/T/ipykernel_21227/2782960600.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("", inplace=True)  # Fill NaN with "Unknown"


In [12]:
 # filtered_df.info()  # Get a summary of the data, including column types and non-null counts
# filtered_df.describe()  # Get statistical summaries of numeric columns
# Display the first few rows to ensure it's loaded correctly
# df.head()

,CI ID,Name,Alias,Approval group,Managed by,Support group,Manufacturer/Vendor,Short Description,Assigned to,Business criticality,...,Product Owner Executive,Product Owner,Revenue Impact,Service Executive,Service Owner,Service Team,Supported by,Updated by,Text,n_tokens
0,CI000007819,Fidelity Retirement Plans and Stock Administr...,Fidelity,mhubHR-ITMarriott-APPRV,Angela Godwin,mhubHR-ITMarriott-Support,,This app is the MI Shares program for some of ...,,4 - not critical,...,,,Revenue Generating,Sanchita Sarkar,Divya Shahani,Corporate Systems,,snair009,CI ID: CI000007819; Name: Fidelity Retirement ...,209
1,CI000539436,4Sight Budgeting and Forecasting Tool,,Global Service Desk-APPRV,Bruce Nguyen,Tier-1-Technical-Support-EG,,,,4 - not critical,...,,,,Naveen Manga,Divya Shahani,Corporate Systems,,cspha088,CI ID: CI000539436; Name: 4Sight Budgeting and...,177
2,CI000553739,Above Property LAN Services,,Network-Datacenter-Cloud-Ops-APPRV,,Network-Datacenter-Cloud-Ops,,Add support services to above-property locatio...,,3 - less critical,...,,,,Matthew Barczak,Allen Shen,Infrastructure – Network,,cspha088,CI ID: CI000553739; Name: Above Property LAN S...,184
3,CI084125824,Accertify - Fraud and Risk,,Accertify-Fraud-and-Risk-check-APPRV,Kathleen Ploof,CorpSys-ITBPP-ITToolsOther-Support-AMER,,,,Tier 2,...,,,,Kathleen Ploof,Catherine Watkins,,,ijess202,CI ID: CI084125824; Name: Accertify - Fraud an...,169
4,BSN0002761,Accessories/Peripherals,Accessories/Peripherals,LAW-SOS-APPRV-AMER,David Sanders,LAWSOS-Support-AMER,,LAWSOS,,Tier 5,...,,,,Gerald Nguyen,David Sanders,,,ijess202,CI ID: BSN0002761; Name: Accessories/Periphera...,160


In [13]:
def create_openai_embeddings(text,input_type="search_document"):
    url = 'https://llmgw.codefest2025.marriott.com/v1/embeddings?model=cohere.embed-english-v3'
    headers = {
        'x-goog-api-key': 'sk-8TzEM3tVNH95RJfD8FRrFg',
        'Content-Type': 'application/json'
    }
    
    data = {
        "input":text,
        "input_type": input_type,
        "texts":text,
    }
    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Request failed with status {response.status_code}")
        print(response.text)
        return None

In [16]:
def chunk_list(lst, chunk_size):
    """Split list into chunks of max chunk_size"""
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

def truncate_text(text, max_chars=8000):
    """Truncate text to avoid token limit issues (roughly estimated)"""
    return text[:max_chars]

def generate_embeddings_safely(texts,input_type):
    all_embeddings = []
    chunk_size = 60  # Cohere model limit
    model_id="cohere.embed-english-v3"
    for batch in chunk_list(texts, chunk_size):
        # Truncate any long text in batch
        batch = [truncate_text(t) for t in batch]
        response = create_openai_embeddings(batch,input_type)
        batch_embeddings = [item['embedding'] for item in response['data']]
        all_embeddings.extend(batch_embeddings)
    
    return all_embeddings

In [17]:
# Function to generate embeddings using OpenAI's API
import numpy as np
def generate_embeddings(text_data,input_type):
    embeddings = create_openai_embeddings(text_data,input_type)

    return [embedding['embedding'] for embedding in embeddings['data']]

# Generate embeddings for the 'Text' column in your DataFrame
embeddings = generate_embeddings_safely(df['Text'].tolist(),"search_document")

# Convert embeddings to a numpy array for use with FAISS
embeddings = np.array(embeddings)

# Optionally, save the embeddings to a file for future use
np.save('embeddings.npy', embeddings)

In [18]:
import faiss

# Initialize a FAISS index (for dense vectors)
dimension = embeddings.shape[1]  # The dimension of your embeddings
index = faiss.IndexFlatL2(dimension)  # L2 distance metric (Euclidean distance)

# Add embeddings to the FAISS index
index.add(embeddings)

# Save the FAISS index to disk for later use
faiss.write_index(index, "faiss_index.index")

# Optionally, save the corresponding DataFrame indices for reference (e.g., to retrieve full rows later)
df['index'] = df.index  # Add the original DataFrame index as a column
df.to_csv("data_with_index.csv", index=False)




In [33]:

def query_vector_database(query, index, df, k=1):
    # Generate embedding for the query
    query_embedding = generate_embeddings([query],"search_query")[0]
    
    # Search for the closest embeddings in the FAISS index (k=1 means the top match)
    D, I = index.search(np.array([query_embedding]), k)  # D is distances, I is indices
    row = (df.iloc[I[0][0]])
    print(row['CI ID'])
    print(df.iloc[I[0][1]])
    print(df.iloc[I[2][2]].to_json())

    # Retrieve the closest match from the DataFrame
    closest_match = df.iloc[I[0][0]]  # Get the row corresponding to the closest match
    return closest_match

# Example query
query = "kabinaya"
result = query_vector_database(query, index, df,5)
print(f"Closest match: {result['Managed by']}")


CI000007953
CI ID                                                           SNSVC0005247
Name                                  Enterprise Event Hub Kafka SAAS - Test
Alias                                                                       
Approval group                            Enterprise-Event-Hub-Support-APPRV
Managed by                                                  Vikram Prabu S P
Support group                            Enterprise-Event-Hub-Support-US-CAN
Manufacturer/Vendor                                                         
Short Description                                                           
Assigned to                                                                 
Business criticality                                                  Tier 5
Class                                          Tag-Based Application Service
Comments                                                                    
Consumer type                                                   

IndexError: index 2 is out of bounds for axis 0 with size 1